### Imports

In [1]:
import os
import random

from datasets import (
    concatenate_datasets,
    Dataset,
    DatasetDict,
    load_dataset,
    load_from_disk,
)
import evaluate
import numpy as np
import pandas as pd
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForPreTraining,
    AutoModel,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments,
)

/home/mateuslatrova/anaconda3/envs/enem/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Variáveis "globais"

In [42]:
checkpoint = "neuralmind/bert-base-portuguese-cased"

raw_files_dirpath = os.path.abspath("data/raw")
preprocessed_files_dirpath = os.path.abspath("data/preprocessed/")
processed_files_dirpath = os.path.abspath("data/processed/")

raw_files_extension = ".csv"

raw_essay_filename = "redacoes"
raw_topic_filename = "temas"

preprocessed_essay_filename = "redacoes_fuga_do_tema"
preprocessed_topic_filename = "temas_fuga_do_tema"

topic_deviation_dataset_filename = "fuga_do_tema"

raw_essay_dataset_filepath = os.path.join(
    raw_files_dirpath, raw_essay_filename + raw_files_extension
)
raw_topic_dataset_filepath = os.path.join(
    raw_files_dirpath, raw_topic_filename + raw_files_extension
)
preprocessed_essay_dataset_filepath = os.path.join(
    preprocessed_files_dirpath, preprocessed_essay_filename
)
preprocessed_topic_dataset_filepath = os.path.join(
    preprocessed_files_dirpath, preprocessed_topic_filename
)

processed_dataset_filepath = os.path.join(processed_files_dirpath, topic_deviation_dataset_filename)

## Exploração do dataset de redações

### Carregamento e primeiro contato com os dados

In [4]:
dataset = load_dataset("csv", data_files=raw_essay_dataset_filepath)

train_dataset = dataset["train"]
train_dataset.set_format("pandas")
train_dataset_df = train_dataset[:]
train_dataset_df.head()

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 91.02it/s]
Generating train split: 385 examples [00:00, 4335.87 examples/s]


,id,id_prompt,title,essay,grades,final_grade,is_ENEM,is_convertible,general,specific
0,0,0,O estudo é a base para melhorar de vida,['Com os avanços que a sociedade deu em tecnol...,"[160, 200, 200, 160, 160]",880,True,True,"Texto muito bom, um pouco prejudicado pelos as...",['1) Há um certo número de problemas de lingua...
1,1,0,O mal do século e o combate através da fé,"['De acordo com pesquisas, o Brasil é o país d...","[120, 120, 120, 160, 120]",640,True,True,Diz um linguista que comunicação é o que o lei...,['1) O texto não apresenta problemas graves de...
2,2,0,Em uma pesquisa recente,['Em uma pesquisa recente realizada pela ONG O...,"[120, 80, 120, 80, 120]",520,True,True,"Texto fraco, que apenas tangencia o tema, dedi...",['1) O autor se expressa com dificuldade. Bast...
3,3,0,O conhecimento liberta,"['A ideia de vencer na vida, presente no imagi...","[160, 200, 160, 200, 160]",880,True,True,"Texto muito bom, com pequenos problemas menore...","['1) Texto muito bem escrito, com alguns probl..."
4,4,0,O caminho do sucesso,"['O caminho do sucesso', 'É de fundamental imp...","[120, 80, 40, 40, 80]",360,False,True,"Lamentavelmente, o texto é muito fraco, em esp...",['1) O autor tem uma dificuldade de se express...


### Validando o formato das redações

In [8]:
samples = train_dataset.shuffle(7).select(range(3))
samples.set_format("pandas")
samples_df = samples[:]
essay_examples = samples_df["essay"].tolist()
essay_examples

["['O carnaval chegou ao Brasil no período colonial assim sendo os escravos pintava os rostos e saiam nas ruas. Tradicionalmente aos católicos é o marco do início da Quaresma 40 dias segue até a sexta-feira santa dois Dias antes da Páscoa.', 'Em primeira análise, o governador Federal libera um valor a cada estado ao incentivo à cultura, empresas privadas destinam partes dos impostos e aplica em eventos da festa.', 'Em segunda análise a brincadeira que encanta diversos turistas de toda nação atrai, também a grandes riscos de vícios e desidratação aos cidadãos no desfile promovendo postos de ambulâncias e policiamento devida a negligencia humana ocasiona mortes acidentais.', 'Por fim, em 2020, a escola de samba de São Clemente no figurino teve o enredo o Presidente da República, embora o carnavalesco pode-se ser usado para manifestar insatisfações sociais.', 'Então, o MINSTÉRIO DA EDUCAÇÃO deve repensar, e assim veria que vale mais gastar o dinheiro da festa carnavalesca com criação de m

In [9]:
# Como há apenas valores "True" nessa máscara, todas as redações iniciam com '[' e terminam com ']'.
mask = (train_dataset_df["essay"].str.startswith("[")) & (
    train_dataset_df["essay"].str.endswith("]")
)
set(mask.values.tolist())

{True}

In [10]:
# Para remover esses caracteres de início e fim:
for example in essay_examples:
    print(example.strip("['']"))

O carnaval chegou ao Brasil no período colonial assim sendo os escravos pintava os rostos e saiam nas ruas. Tradicionalmente aos católicos é o marco do início da Quaresma 40 dias segue até a sexta-feira santa dois Dias antes da Páscoa.', 'Em primeira análise, o governador Federal libera um valor a cada estado ao incentivo à cultura, empresas privadas destinam partes dos impostos e aplica em eventos da festa.', 'Em segunda análise a brincadeira que encanta diversos turistas de toda nação atrai, também a grandes riscos de vícios e desidratação aos cidadãos no desfile promovendo postos de ambulâncias e policiamento devida a negligencia humana ocasiona mortes acidentais.', 'Por fim, em 2020, a escola de samba de São Clemente no figurino teve o enredo o Presidente da República, embora o carnavalesco pode-se ser usado para manifestar insatisfações sociais.', 'Então, o MINSTÉRIO DA EDUCAÇÃO deve repensar, e assim veria que vale mais gastar o dinheiro da festa carnavalesca com criação de mais 

In [11]:
# Além disso, aparentemente, os parágrafos da redação são iniciados e terminados pelos caracteres "\'" ou apenas "'".
# Para validar isso, faremos o seguinte teste:
for example in essay_examples:
    example = example.strip("['']")
    removed_quote_and_comma = example.replace("',", "")
    removed_quote = removed_quote_and_comma.replace("'", "")
    removed_slash = removed_quote.replace("\\", "")
    print(removed_slash)

O carnaval chegou ao Brasil no período colonial assim sendo os escravos pintava os rostos e saiam nas ruas. Tradicionalmente aos católicos é o marco do início da Quaresma 40 dias segue até a sexta-feira santa dois Dias antes da Páscoa. Em primeira análise, o governador Federal libera um valor a cada estado ao incentivo à cultura, empresas privadas destinam partes dos impostos e aplica em eventos da festa. Em segunda análise a brincadeira que encanta diversos turistas de toda nação atrai, também a grandes riscos de vícios e desidratação aos cidadãos no desfile promovendo postos de ambulâncias e policiamento devida a negligencia humana ocasiona mortes acidentais. Por fim, em 2020, a escola de samba de São Clemente no figurino teve o enredo o Presidente da República, embora o carnavalesco pode-se ser usado para manifestar insatisfações sociais. Então, o MINSTÉRIO DA EDUCAÇÃO deve repensar, e assim veria que vale mais gastar o dinheiro da festa carnavalesca com criação de mais escolas com 

In [12]:
# Um jeito mais eficiente que descobri para realizar a mesma tarefa:
for example in essay_examples:
    essay = ""
    for paragraph in eval(example):
        essay += paragraph
    print(essay)

O carnaval chegou ao Brasil no período colonial assim sendo os escravos pintava os rostos e saiam nas ruas. Tradicionalmente aos católicos é o marco do início da Quaresma 40 dias segue até a sexta-feira santa dois Dias antes da Páscoa.Em primeira análise, o governador Federal libera um valor a cada estado ao incentivo à cultura, empresas privadas destinam partes dos impostos e aplica em eventos da festa.Em segunda análise a brincadeira que encanta diversos turistas de toda nação atrai, também a grandes riscos de vícios e desidratação aos cidadãos no desfile promovendo postos de ambulâncias e policiamento devida a negligencia humana ocasiona mortes acidentais.Por fim, em 2020, a escola de samba de São Clemente no figurino teve o enredo o Presidente da República, embora o carnavalesco pode-se ser usado para manifestar insatisfações sociais.Então, o MINSTÉRIO DA EDUCAÇÃO deve repensar, e assim veria que vale mais gastar o dinheiro da festa carnavalesca com criação de mais escolas com qual

### Validando o que significa o campo "id_prompt"

In [13]:
# Validando se o campo "id_prompt" é de fato um identificador para o tema da redação
id_prompt_to_count = train_dataset_df["id_prompt"].value_counts().to_dict()
list(id_prompt_to_count.items())[0:10]

[(0, 20),
 (15, 20),
 (40, 20),
 (39, 20),
 (37, 20),
 (35, 20),
 (34, 20),
 (33, 20),
 (27, 20),
 (1, 20)]

In [14]:
# Observando o dataframe abaixo, podemos concluir que, de fato, o campo "id_prompt" é um identificador para
# o tema da redação.
train_dataset_df[train_dataset_df["id_prompt"] == 1].sample(frac=1).head(10)

,id,id_prompt,title,essay,grades,final_grade,is_ENEM,is_convertible,general,specific
36,36,1,Remédio para a falta de responsabilidade,['Os resultados das urnas nas últimas eleições...,"[200, 160, 160, 200, 120]",840,True,True,Texto muito bom. Satisfaz plenamente as compet...,[]
33,33,1,O futuro é agora: politize-se!,"['O Brasil carrega desde o passado, caracterís...","[80, 80, 80, 80, 80]",400,True,True,"Lamentavelmente, o texto é muito fraco. As úni...",['1) Primeiro parágrafo: a frase em vermelho n...
29,29,1,"Onda da justiça, onda do bem",['A onda conservadora tomou proporções signifi...,"[120, 80, 80, 80, 80]",440,True,True,Texto fraco. Exprime apenas o entusiasmo do au...,['1) Primeiro parágrafo: a introdução é fraca ...
28,28,1,A Volta da Política Conservadora no Brasil,"['O Brasil vive uma nova fase na política, com...","[40, 80, 120, 40, 0]",280,False,True,"Infelizmente, o texto é muito fraco. O autor n...",['1) Primeiro parágrafo: a) redundância: “a el...
21,21,1,O conservadorismo no Brasil (2),"['O conservadorismo, que vem ganhando espaço n...","[120, 120, 120, 120, 80]",560,True,True,"Texto mediano, em que falta desenvolvimento e ...",['Segundo parágrafo: a) o uso incorreto da pal...
26,26,1,O conservadorismo no Brasil,['A onda conservadora no Brasil alcançou um el...,"[120, 80, 80, 80, 40]",400,False,True,"Texto fraco. É, na verdade, um esquema que pre...",['1) Primeiro parágrafo: o que o autor entende...
31,31,1,Nada que preste,"['Na França, em 1789 à 1799 ocorreu a sua revo...","[160, 80, 40, 80, 40]",400,False,True,"Texto fraco. Infelizmente, o autor não persegu...",['1) Primeiro e segundo parágrafo: conquanto o...
25,25,1,O homem é naturalmente bom,['O conservadorismo está em voga na contempora...,"[160, 160, 120, 160, 160]",760,True,True,"Texto bom, com poucos deslizes e erros de ling...",[]
22,22,1,Um novo olhar sobre o Brasil,['O nosso país está vivenciando diversos acont...,"[0, 0, 0, 0, 0]",0,True,True,"Infelizmente, a redação é fraca. No entanto, e...",['1) Primeiro parágrafo: a introdução é muito ...
32,32,1,A onda conservadora e o Brasil nos próximos anos,['O país está em transição para um novo govern...,"[40, 40, 40, 40, 40]",200,False,True,"Lamentavelmente, o texto só pode ser considera...",['1) Primeiro parágrafo: a) pela ordem dos ter...


### Validando remoção de colunas sem dados

#### Obs.: no novo dataset, essas colunas possuem dados, então não será necessário removê-las

In [12]:
# Como todos valores dessas duas colunas são nulos, podemos removê-las.
columns_to_be_removed = ["general", "specific"]
for column in columns_to_be_removed:
    print(all(train_dataset_df[column].isna()))

True
True


### Validando se é possível remover valores estranhos no campo "title"

#### Obs.: no novo dataset, os valores dessa coluna já foram limpos.

In [16]:
title_to_count = train_dataset_df["title"].value_counts()
title_to_count = {title: count for title, count in title_to_count.items() if count > 1}
title_to_count

{}

In [17]:
# Depois de analisar todos esses títulos estranhos, concluiu-se que todos eles são títulos
# inválidos, ou são simplesmente um valor indicando que a redação não teve título.
suspicious_titles = list(title_to_count.keys())
train_dataset_df[train_dataset_df["title"].isin(suspicious_titles)].sort_values(by=["title"])

,id,id_prompt,title,essay,grades,final_grade,is_ENEM,is_convertible,general,specific


In [17]:
# Portanto, vamos apenas colocar um valor nulo para todos esses títulos inválidos
train_dataset_df.loc[train_dataset_df["title"].isin(suspicious_titles), "title"] = pd.NA
train_dataset_df[train_dataset_df["title"].isin(suspicious_titles)]

,id,id_prompt,title,essay,grades,general,specific


In [18]:
train_dataset_df["title"].unique().tolist()

['O estudo é a base para melhorar de vida',
 'O mal do século e o combate através da fé',
 'Em uma pesquisa recente',
 'O conhecimento liberta',
 'O caminho do sucesso',
 'Nem tudo é dinheiro',
 'Desejo pelo Sucesso',
 'Estudar',
 'O que realmente é importante para a melhoria de vida',
 'Fé: essencial para a vida em geral ou para vida espiritual',
 'A fé é o equilíbrio do brasileiro',
 'A fé ao encontro da ciência',
 'A arte do equilíbrio da vida',
 'Ser vencedor é uma escolha',
 'Povo que anda com fé',
 'Vencer na vida',
 'Vencer ou ser?',
 '“E essa é a vitória que vence o mundo: a nossa fé”',
 'O que é mais importante para vencer na vida?',
 'Fatores mais importantes para vencer na vida',
 'Por que uma onda conservadora?',
 'O conservadorismo no Brasil (2)',
 'Um novo olhar sobre o Brasil',
 'A nova hegemonia política do Brasil',
 'O que falta ao Brasil para prosperar?',
 'O homem é naturalmente bom',
 'O conservadorismo no Brasil',
 'A renovação do Brasil',
 'A Volta da Política Con

Porém, depois de analisar mais exemplos, percebeu-se que há mais títulos inválidos. Então, caso seja realmente necessário remover esses títulos inválidos para alguma tarefa, isso será realizado posteriormente devido ao grande número de exemplos.

### Análise do campo "grades"

In [19]:
samples = train_dataset.shuffle(7).select(range(3))
samples.set_format("pandas")
samples_df = samples[:]
grades_examples = samples_df["grades"].tolist()
grades_examples

['[0, 0, 0, 0, 0]', '[200, 120, 120, 120, 160]', '[120, 120, 120, 120, 120]']

In [20]:
type(grades_examples[0])

str

In [21]:
example = grades_examples[0]
grades_without_brackets = example.strip("[]")
grades_without_brackets

'0, 0, 0, 0, 0'

In [22]:
# Obtendo a nota de cada competência separadamente, podemos criar uma feature de nota por competência, sendo do tipo inteiro.
[int(grade) for grade in grades_without_brackets.split(",")]

[0, 0, 0, 0, 0]

In [23]:
# Uma forma mais eficiente de realizar a mesma tarefa acima:
for example in grades_examples:
    print(eval(example))

[0, 0, 0, 0, 0]
[200, 120, 120, 120, 160]
[120, 120, 120, 120, 120]


## Exploração do dataset de temas

In [156]:
df = pd.read_csv(raw_topic_dataset_filepath)
df.head()

,id,source,title,prompt,supporting_text
0,0,UOL,O que é mais importante para vencer na vida?,['Reportagem publicada pelo UOL Economia no mê...,"['Pessoas valorizam fé, em vez de estudo e tra..."
1,1,UOL,A onda conservadora e o Brasil nos próximos an...,"['Em meados desta década, que se aproxima do f...","['A nova onda conservadora no Brasil', 'Após e..."
2,2,UOL,A terapia de reversão da orientação sexual,['Um assunto sensível transformou-se em notíci...,"['Os fatos', 'Uma decisão em caráter liminar d..."
3,3,UOL,Agrotóxicos ou defensivos agrícolas: dois nome...,"['O fato é recente, mas o debate já dura mais ...","['Brasil, potência agrícola', 'O Governo do pr..."
4,4,UOL,Artes e educação física: opcionais ou obrigató...,"['No mês de setembro passado, o governo federa...","['Educação integral', 'Ao justificar a reforma..."


### Entendendo a coluna "prompt" (descrição do tema)

In [158]:
topic_example = df.loc[0, "prompt"]
topic_example

"['Reportagem publicada pelo UOL Economia no mês passado apresenta uma pesquisa realizada pelo Datafolha em que se perguntava aos entrevistados qual o fator mais importante para se conquistar uma vida melhor. A amostragem reflete toda a população do Brasil, com baixa margem de erro. Os resultados da pesquisa revelaram que, em primeiro lugar, os brasileiros consideram necessária a fé religiosa. De acordo com a pesquisa, as pessoas consideram a fé mais importante do que o estudo ou o trabalho, por exemplo, para melhorar de vida. Leia o texto do UOL que se transcreve abaixo, preste atenção nos percentuais e redija uma dissertação argumentativa apresentando sua opinião sobre a questão formulada na pesquisa: para você, dos itens mencionados, qual é o mais importante para melhorar de vida? Por quê? Apresente suas razões para justificar o seu ponto de vista.']"

In [159]:
eval(topic_example)

['Reportagem publicada pelo UOL Economia no mês passado apresenta uma pesquisa realizada pelo Datafolha em que se perguntava aos entrevistados qual o fator mais importante para se conquistar uma vida melhor. A amostragem reflete toda a população do Brasil, com baixa margem de erro. Os resultados da pesquisa revelaram que, em primeiro lugar, os brasileiros consideram necessária a fé religiosa. De acordo com a pesquisa, as pessoas consideram a fé mais importante do que o estudo ou o trabalho, por exemplo, para melhorar de vida. Leia o texto do UOL que se transcreve abaixo, preste atenção nos percentuais e redija uma dissertação argumentativa apresentando sua opinião sobre a questão formulada na pesquisa: para você, dos itens mencionados, qual é o mais importante para melhorar de vida? Por quê? Apresente suas razões para justificar o seu ponto de vista.']

In [160]:
type(eval(topic_example))

list

In [161]:
topic_text = ""
for paragraph in eval(topic_example):
    if not paragraph.endswith("."):
        paragraph = paragraph + "."
    topic_text += paragraph
print(type(topic_text))
topic_text

<class 'str'>


'Reportagem publicada pelo UOL Economia no mês passado apresenta uma pesquisa realizada pelo Datafolha em que se perguntava aos entrevistados qual o fator mais importante para se conquistar uma vida melhor. A amostragem reflete toda a população do Brasil, com baixa margem de erro. Os resultados da pesquisa revelaram que, em primeiro lugar, os brasileiros consideram necessária a fé religiosa. De acordo com a pesquisa, as pessoas consideram a fé mais importante do que o estudo ou o trabalho, por exemplo, para melhorar de vida. Leia o texto do UOL que se transcreve abaixo, preste atenção nos percentuais e redija uma dissertação argumentativa apresentando sua opinião sobre a questão formulada na pesquisa: para você, dos itens mencionados, qual é o mais importante para melhorar de vida? Por quê? Apresente suas razões para justificar o seu ponto de vista.'

### Entendendo a coluna "supporting text" (texto motivador sobre o tema)

In [17]:
supporting_text_example = df.loc[0, "supporting_text"]
supporting_text_example

"['Pessoas valorizam fé, em vez de estudo e trabalho, para melhorar de vida', 'A fé religiosa é o aspecto mais importante para melhorar de vida para 28% dos brasileiros, segundo uma pesquisa divulgada pela ONG Oxfam Brasil. Esse percentual supera até mesmo aqueles que consideram os estudos (21%) , o trabalho (11%) e ganhar mais dinheiro (8%) como o mais importante para ter uma vida melhor. Ter acesso à saúde foi citado como o mais importante para a melhoria de vida de 19% dos entrevistados na pesquisa da Oxfam, ocupando a terceira posição. Outros itens apontados na pesquisa como prioritários para a melhoria de vida foram: ter acesso à aposentadoria (6%) , apoio financeiro da família (5%) e cultura e lazer (2%) . Veja abaixo o que os brasileiros consideram mais importante para melhorar de vida, segundo a Oxfam: - Fé religiosa: 28% - Estudar: 21% - Ter acesso a atendimento de saúde: 19% - Crescer no trabalho: 11% - Ganhar mais dinheiro: 8% - Ter acesso à aposentadoria: 6% - Apoio finance

In [20]:
eval(supporting_text_example)

['Pessoas valorizam fé, em vez de estudo e trabalho, para melhorar de vida',
 'A fé religiosa é o aspecto mais importante para melhorar de vida para 28% dos brasileiros, segundo uma pesquisa divulgada pela ONG Oxfam Brasil. Esse percentual supera até mesmo aqueles que consideram os estudos (21%) , o trabalho (11%) e ganhar mais dinheiro (8%) como o mais importante para ter uma vida melhor. Ter acesso à saúde foi citado como o mais importante para a melhoria de vida de 19% dos entrevistados na pesquisa da Oxfam, ocupando a terceira posição. Outros itens apontados na pesquisa como prioritários para a melhoria de vida foram: ter acesso à aposentadoria (6%) , apoio financeiro da família (5%) e cultura e lazer (2%) . Veja abaixo o que os brasileiros consideram mais importante para melhorar de vida, segundo a Oxfam: - Fé religiosa: 28% - Estudar: 21% - Ter acesso a atendimento de saúde: 19% - Crescer no trabalho: 11% - Ganhar mais dinheiro: 8% - Ter acesso à aposentadoria: 6% - Apoio finance

In [21]:
type(eval(supporting_text_example))

list

In [28]:
supporting_text = ""
for paragraph in eval(supporting_text_example):
    if not paragraph.endswith("."):
        paragraph = paragraph + "."
    supporting_text += paragraph
print(type(supporting_text))
supporting_text

<class 'str'>


'Pessoas valorizam fé, em vez de estudo e trabalho, para melhorar de vida.A fé religiosa é o aspecto mais importante para melhorar de vida para 28% dos brasileiros, segundo uma pesquisa divulgada pela ONG Oxfam Brasil. Esse percentual supera até mesmo aqueles que consideram os estudos (21%) , o trabalho (11%) e ganhar mais dinheiro (8%) como o mais importante para ter uma vida melhor. Ter acesso à saúde foi citado como o mais importante para a melhoria de vida de 19% dos entrevistados na pesquisa da Oxfam, ocupando a terceira posição. Outros itens apontados na pesquisa como prioritários para a melhoria de vida foram: ter acesso à aposentadoria (6%) , apoio financeiro da família (5%) e cultura e lazer (2%) . Veja abaixo o que os brasileiros consideram mais importante para melhorar de vida, segundo a Oxfam: - Fé religiosa: 28% - Estudar: 21% - Ter acesso a atendimento de saúde: 19% - Crescer no trabalho: 11% - Ganhar mais dinheiro: 8% - Ter acesso à aposentadoria: 6% - Apoio financeiro d

### Entendendo a coluna "title" (título do tema)

In [78]:
title_example = df.loc[0, "title"]
print(title_example)

O que é mais importante para vencer na vida?  


In [79]:
print(type(title_example))

<class 'str'>


## Pré-processamento das redações para detecção de fuga ao tema

In [14]:
class EssayDatasetPreprocessorForTopicDeviation:
    COLUMNS_TO_REMOVE = [
        "title",
        "final_grade",
        "is_ENEM",
        "is_convertible",
        "general",
        "specific",
        "grades",
    ]
    RENAME_COLUMNS_MAP = {
        "id": "essay_id",
        "id_prompt": "topic_id",
        "essay": "essay_text",
    }
    SECOND_GRADE_INDEX = 1
    POSITIVE_LABEL = 1
    NEGATIVE_LABEL = 0
    GRADE_THRESHOLD = 0
    TEST_DATASET_SIZE = 0.15
    ESSAY_ID_COLUMN = "essay_id"

    def __init__(self, dataset: Dataset):
        self.original_dataset = dataset

    def preprocess_dataset(self) -> DatasetDict:
        self.train_dataset = (
            self.original_dataset.rename_columns(self.RENAME_COLUMNS_MAP)
            .map(self._preprocess_essay)
            .map(self._create_column_for_topic_deviation_label)
            .map(lambda example: self._set_is_artificial_value(False, example))
        )
        self.train_dataset = self._create_column_to_categorize_artificial_examples(
            self.train_dataset, is_artificial=False
        )

        self.artificial_negative_examples_dataset = (
            self._create_artificial_negative_examples_dataset()
        )
        self.artificial_negative_examples_dataset = (
            self._create_column_to_categorize_artificial_examples(
                self.artificial_negative_examples_dataset, is_artificial=True
            )
        )

        self.train_dataset = concatenate_datasets(
            [self.train_dataset, self.artificial_negative_examples_dataset]
        ).remove_columns(self.COLUMNS_TO_REMOVE)

        self.test_dataset = self._create_test_dataset()

        self.train_dataset = self._remove_rows_from_dataset(
            rows_to_remove=self.test_dataset, dataset_to_remove_from=self.train_dataset
        )

        self.dataset_dict = DatasetDict({"train": self.train_dataset, "test": self.test_dataset})

        return self.dataset_dict

    def _preprocess_essay(self, example):
        essay_before = example["essay_text"]
        essay_after = ""

        paragraphs = eval(essay_before)
        for paragraph in paragraphs:
            essay_after += paragraph

        example["essay_text"] = essay_after
        return example

    def _create_column_for_topic_deviation_label(self, example):
        grades_list = eval(example["grades"])
        second_grade = grades_list[self.SECOND_GRADE_INDEX]

        if second_grade > self.GRADE_THRESHOLD:
            example["label"] = self.POSITIVE_LABEL
        else:
            example["label"] = self.NEGATIVE_LABEL

        return example

    def _create_column_to_categorize_artificial_examples(self, dataset, is_artificial):
        return dataset.map(lambda example: self._set_is_artificial_value(is_artificial, example))

    def _set_is_artificial_value(self, is_artificial, example):
        example["is_artificial"] = is_artificial
        return example

    def _create_artificial_negative_examples_dataset(self):
        positive_examples_dataset = self.train_dataset.filter(
            lambda example: example["label"] == self.POSITIVE_LABEL
        )
        negative_examples_dataset = self.train_dataset.filter(
            lambda example: example["label"] == self.NEGATIVE_LABEL
        )

        positive_examples_dataset.set_format("pandas")

        positive_examples_df = positive_examples_dataset[:]

        number_of_artificial_examples = len(positive_examples_dataset) - len(
            negative_examples_dataset
        )

        artificial_negative_examples_df = positive_examples_df.sample(
            n=number_of_artificial_examples
        )

        min_topic_id = min(self.train_dataset["topic_id"])
        max_topic_id = max(self.train_dataset["topic_id"])

        artificial_negative_examples_df["topic_id"] = artificial_negative_examples_df[
            "topic_id"
        ].apply(
            lambda topic_id: self._random_integer_with_blacklist(
                min_topic_id, max_topic_id, [topic_id]
            )
        )
        artificial_negative_examples_df["label"] = self.NEGATIVE_LABEL

        artificial_negative_examples_dataset = Dataset.from_pandas(
            artificial_negative_examples_df, preserve_index=False
        )

        return artificial_negative_examples_dataset

    def _random_integer_with_blacklist(self, min_val, max_val, blacklist):
        while True:
            random_num = random.randint(min_val, max_val)
            if random_num not in blacklist:
                return random_num

    def _create_test_dataset(self):
        original_negative_examples = self.train_dataset.filter(
            lambda example: example["label"] == self.NEGATIVE_LABEL
            and example["is_artificial"] == False
        )
        artificial_negative_examples = self.train_dataset.filter(
            lambda example: example["label"] == self.NEGATIVE_LABEL
            or example["is_artificial"] == True
        )
        positive_examples = self.train_dataset.filter(
            lambda example: example["label"] == self.POSITIVE_LABEL
        )

        num_of_original_negative_examples = 10
        test_original_negative_examples = original_negative_examples.shuffle().select(
            range(num_of_original_negative_examples)
        )

        num_of_negative_examples = num_of_positive_examples = int(
            self.TEST_DATASET_SIZE * len(train_dataset) / 2
        )
        num_of_missing_negative_examples = (
            num_of_negative_examples - num_of_original_negative_examples
        )

        test_artificial_negative_examples = artificial_negative_examples.shuffle().select(
            range(num_of_missing_negative_examples)
        )

        test_positive_examples = positive_examples.shuffle().select(range(num_of_positive_examples))

        test_dataset = concatenate_datasets(
            [
                test_positive_examples,
                test_artificial_negative_examples,
                test_original_negative_examples,
            ]
        ).shuffle()

        return test_dataset

    def _remove_rows_from_dataset(self, rows_to_remove: Dataset, dataset_to_remove_from: Dataset):
        ids_to_remove = set(rows_to_remove[self.ESSAY_ID_COLUMN])
        return dataset_to_remove_from.filter(
            lambda example: self._keep_example_if_not_in_remove_list(ids_to_remove, example)
        )

    def _keep_example_if_not_in_remove_list(self, ids_to_remove, example):
        return example[self.ESSAY_ID_COLUMN] not in ids_to_remove

In [7]:
dataset_dict = load_dataset("csv", data_files=raw_essay_dataset_filepath)
train_dataset = dataset_dict["train"]
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['id', 'id_prompt', 'title', 'essay', 'grades', 'final_grade', 'is_ENEM', 'is_convertible', 'general', 'specific'],
        num_rows: 385
    })
})

In [9]:
train_dataset = dataset_dict["train"]
train_dataset

Dataset({
    features: ['id', 'id_prompt', 'title', 'essay', 'grades', 'final_grade', 'is_ENEM', 'is_convertible', 'general', 'specific'],
    num_rows: 385
})

In [15]:
preprocessor = EssayDatasetPreprocessorForTopicDeviation(train_dataset)
preprocessed_dataset_dict = preprocessor.preprocess_dataset()
preprocessed_dataset_dict

Filter: 100%|██████████| 656/656 [00:00<00:00, 28320.06 examples/s]


DatasetDict({
    train: Dataset({
        features: ['essay_id', 'topic_id', 'essay_text', 'label', 'is_artificial'],
        num_rows: 567
    })
    test: Dataset({
        features: ['essay_id', 'topic_id', 'essay_text', 'label', 'is_artificial'],
        num_rows: 56
    })
})

In [16]:
# Save preprocessed dataset for later use:
preprocessed_dataset_dict.save_to_disk())

Saving the dataset (1/1 shards): 100%|██████████| 56/56 [00:00<00:00, 3643.49 examples/s]


## Pré-processamento do dataset de temas para detecção de fuga ao tema

In [17]:
class TopicDatasetPreprocessorForTopicDeviation:
    COLUMNS_TO_REMOVE = ["source", "title", "prompt", "supporting_text"]
    RENAME_COLUMNS_MAP = {
        "id": "topic_id",
    }
    PARAGRAPH_ENDINGS = [".", "?", "!"]

    def __init__(self, dataset: Dataset):
        self.dataset = dataset

    def preprocess_dataset(self):
        self.preprocessed_dataset = (
            self.dataset.rename_columns(self.RENAME_COLUMNS_MAP)
            .map(self._concatenate_texts)
            .remove_columns(self.COLUMNS_TO_REMOVE)
        )
        return self.preprocessed_dataset

    def _concatenate_texts(self, example):
        concatenated_texts = ""

        topic = example["prompt"]
        supporting_text = example["supporting_text"]

        # This try-catch block is necessary because some titles have quotes around it,
        # but others don't.
        try:
            title = eval(example["title"])
        except SyntaxError as error:
            title = example["title"]

        paragraphs = [title] + eval(supporting_text) + eval(topic)

        for paragraph in paragraphs:
            has_desired_ending = False
            for ending in self.PARAGRAPH_ENDINGS:
                if paragraph.endswith(ending):
                    has_desired_ending = True
                    break

            if not has_desired_ending:
                paragraph = paragraph + "."

            concatenated_texts += paragraph

        example["topic_text"] = concatenated_texts
        return example

In [18]:
dataset_dict = load_dataset("csv", data_files=raw_topic_dataset_filepath)
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['id', 'source', 'title', 'prompt', 'supporting_text'],
        num_rows: 44
    })
})

In [19]:
train_dataset = dataset_dict["train"]
train_dataset

Dataset({
    features: ['id', 'source', 'title', 'prompt', 'supporting_text'],
    num_rows: 44
})

In [20]:
preprocessor = TopicDatasetPreprocessorForTopicDeviation(train_dataset)
preprocessed_topic_dataset = preprocessor.preprocess_dataset()
preprocessed_topic_dataset

Map: 100%|██████████| 44/44 [00:00<00:00, 1604.29 examples/s]


Dataset({
    features: ['topic_id', 'topic_text'],
    num_rows: 44
})

In [21]:
samples = preprocessed_topic_dataset.select([0, 5, 10])
for sample in samples:
    print(sample["topic_text"])

O que é mais importante para vencer na vida?  .Pessoas valorizam fé, em vez de estudo e trabalho, para melhorar de vida.A fé religiosa é o aspecto mais importante para melhorar de vida para 28% dos brasileiros, segundo uma pesquisa divulgada pela ONG Oxfam Brasil. Esse percentual supera até mesmo aqueles que consideram os estudos (21%) , o trabalho (11%) e ganhar mais dinheiro (8%) como o mais importante para ter uma vida melhor. Ter acesso à saúde foi citado como o mais importante para a melhoria de vida de 19% dos entrevistados na pesquisa da Oxfam, ocupando a terceira posição. Outros itens apontados na pesquisa como prioritários para a melhoria de vida foram: ter acesso à aposentadoria (6%) , apoio financeiro da família (5%) e cultura e lazer (2%) . Veja abaixo o que os brasileiros consideram mais importante para melhorar de vida, segundo a Oxfam: - Fé religiosa: 28% - Estudar: 21% - Ter acesso a atendimento de saúde: 19% - Crescer no trabalho: 11% - Ganhar mais dinheiro: 8% - Ter a

In [24]:
# Save preprocessed dataset for later use:
dataset_dict["train"] = preprocessed_topic_dataset
dataset_dict.save_to_disk(preprocessed_topic_dataset_filepath)

Saving the dataset (1/1 shards): 100%|██████████| 44/44 [00:00<00:00, 3708.86 examples/s]


## Combinação do dataset de redações com o de temas

In [35]:
class EssayAndTopicDatasetsCombinator:
    COLUMNS_ORDER = ["essay_id", "essay_text", "topic_id", "topic_text", "label"]
    SPLITS = ["test", "train"]

    def combine(self, essay_dataset_dict: DatasetDict, topic_dataset: Dataset) -> DatasetDict:
        topic_dataset.set_format("pandas")
        topic_df = topic_dataset[:].set_index("topic_id")
        combined_dataset_dict = {}

        for split in self.SPLITS:
            essay_dataset = essay_dataset_dict[split]
            essay_dataset.set_format("pandas")

            essay_df = essay_dataset[:]

            combined_df = essay_df.join(topic_df, on="topic_id")
            combined_df = combined_df.loc[:, self.COLUMNS_ORDER]

            essay_dataset.set_format()

            combined_dataset = Dataset.from_pandas(combined_df, preserve_index=False)

            combined_dataset_dict[split] = combined_dataset

        combined_dataset_dict = DatasetDict(combined_dataset_dict)

        return combined_dataset_dict

In [36]:
essay_dataset_dict = load_from_disk(preprocessed_essay_dataset_filepath)
essay_dataset_dict

DatasetDict({
    train: Dataset({
        features: ['essay_id', 'topic_id', 'essay_text', 'label', 'is_artificial'],
        num_rows: 567
    })
    test: Dataset({
        features: ['essay_id', 'topic_id', 'essay_text', 'label', 'is_artificial'],
        num_rows: 56
    })
})

In [37]:
topic_dataset = load_from_disk(preprocessed_topic_dataset_filepath)["train"]
topic_dataset

Dataset({
    features: ['topic_id', 'topic_text'],
    num_rows: 44
})

In [38]:
combinator = EssayAndTopicDatasetsCombinator()
combinator

In [40]:
combined_dataset_dict = combinator.combine(essay_dataset_dict, topic_dataset)
combined_dataset_dict

DatasetDict({
    test: Dataset({
        features: ['essay_id', 'essay_text', 'topic_id', 'topic_text', 'label'],
        num_rows: 56
    })
    train: Dataset({
        features: ['essay_id', 'essay_text', 'topic_id', 'topic_text', 'label'],
        num_rows: 567
    })
})

In [44]:
# Save processed dataset for later use:
combined_dataset_dict.save_to_disk(processed_dataset_filepath)

Saving the dataset (1/1 shards): 100%|██████████| 567/567 [00:00<00:00, 53591.36 examples/s]


## Exemplo da tokenização de uma redação

In [45]:
essay_dataset_dict = load_from_disk(preprocessed_essay_dataset_filepath)
essay_dataset_dict

DatasetDict({
    train: Dataset({
        features: ['essay_id', 'topic_id', 'essay_text', 'label', 'is_artificial'],
        num_rows: 567
    })
    test: Dataset({
        features: ['essay_id', 'topic_id', 'essay_text', 'label', 'is_artificial'],
        num_rows: 56
    })
})

In [46]:
essay_train_dataset = essay_dataset_dict["train"]
essay_examples = essay_train_dataset.shuffle(23).select(range(3))["essay_text"]
for example in essay_examples:
    print(example)

Em 2014, durante o governo Dilma Rousseff, iniciava a operação Lava Jato que tinha o objetivo de investigar casos de corrupção envolvendo empresas estatais, como: a Petrobras. Em virtude disso, várias notícias relatando a participação de políticos em esquemas fraudulentos começaram a surgir e causar indignação na população. Por conseguinte, o anseio da sociedade em acompanhar os desdobramentos desses fatos ocasionaram a notoriedade do Supremo Tribunal Federal.Mormente, ao avaliar que, o papel desse órgão judiciário, consiste na guarda da Constituição Federal e no julgamento de ocorrências concernentes ao Congresso Nacional ou há casos que sejam relevantes para todo o país. Nessa conjectura, é indiscutível a importância das atribuições do Supremo Tribunal Federal, visto que algumas pautas, como: a criminalização da homofobia, trouxeram grandes avanços para a sociedade. Entretanto, processos envolvendo ex-presidentes, ministros e ex-assessores ganharam destaque em âmbito nacional devido 

In [47]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint, do_lower_case=False)
tokenizer

BertTokenizerFast(name_or_path='neuralmind/bert-base-portuguese-cased', vocab_size=29794, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [48]:
tokenized_essays = []
for essay in essay_examples:
    tokenized_essay = tokenizer.tokenize([essay], is_split_into_words=True)
    tokenized_essays.append(tokenized_essay)
tokenized_essays[0][:20]

['Em',
 '2014',
 ',',
 'durante',
 'o',
 'governo',
 'Dilma',
 'Rousseff',
 ',',
 'inicia',
 '##va',
 'a',
 'operação',
 'Lava',
 'Jato',
 'que',
 'tinha',
 'o',
 'objetivo',
 'de']

## Fine-tuning para classificar nota em relação à abordagem do tema da redação

In [50]:
dataset_dict = load_from_disk(processed_dataset_filepath)
train_dataset = dataset_dict["train"]
test_dataset = dataset_dict["test"]
print(train_dataset)
print(test_dataset)

Dataset({
    features: ['essay_id', 'essay_text', 'topic_id', 'topic_text', 'label'],
    num_rows: 567
})
Dataset({
    features: ['essay_id', 'essay_text', 'topic_id', 'topic_text', 'label'],
    num_rows: 56
})


In [51]:
def tokenize_function(example):
    inputs = tokenizer(
        example["essay_text"],
        example["topic_text"],
        padding=True,
        truncation=True,
        max_length=512,
        add_special_tokens=True,
        return_tensors="pt",
    )
    labels = torch.tensor(example["label"])
    return inputs, labels

In [52]:
tokenized_train_data = train_dataset.map(tokenize_function, batched=True, batch_size=32)
tokenized_test_data = test_dataset.map(tokenize_function, batched=True, batch_size=32)
print(tokenized_train_data)
print(tokenized_test_data)

Map:   0%|          | 0/567 [00:00<?, ? examples/s]

Map: 100%|██████████| 56/56 [00:00<00:00, 277.34 examples/s]

Dataset({
    features: ['essay_id', 'essay_text', 'topic_id', 'topic_text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 567
})
Dataset({
    features: ['essay_id', 'essay_text', 'topic_id', 'topic_text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 56
})


In [54]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [55]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="topic_deviation_model",
    per_device_train_batch_size=32,
    num_train_epochs=3,
    evaluation_strategy="epoch",
)

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_train_data,
    eval_dataset=tokenized_test_data,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [209]:
trainer.train()

ImportError: To be able to use evaluate-metric/glue, you need to install the following dependencies['sklearn', 'scipy'] using 'pip install sklearn scipy' for instance'

In [ ]:
trainer.evaluate()

In [ ]:
trainer.save_model()